# Point-to-Point space charge example

This uses a basic APEX Gun example with a reduced voltage and artificially reduced frequency. 

Warning from Ji: The # of electrons divided by the number of processors should be an integer.


In [ ]:
from impact import Impact
from distgen import Generator
import os

from pmd_beamphysics.units import e_charge, mec2

import matplotlib.pyplot as plt

In [ ]:
IMPACT_IN = "templates/apex_gun/ImpactT.in"
os.path.exists(IMPACT_IN)

# Prepare particles

Prepare 100 electrons in a 1 µm uniform spot.

In [ ]:
DISTGEN_IN = """
n_particle: 100
species: electron
r_dist:
  max_r: 1000 nm
  type: radial_uniform
random:
  type: hammersley
start:
  MTE: 30 meV
  type: cathode
t_dist:
  avg_t: 0 fs
  sigma_t: 29.0 fs
  type: gaussian
total_charge: 1.602176634e-17 C
"""

In [ ]:
G = Generator(DISTGEN_IN)
G["n_particle"] = 100
# Set individual electrons
G["total_charge:value"] = e_charge * G["n_particle"]

G.run()
P0 = G.particles

fig, ax = plt.subplots()
ax.scatter(P0.x * 1e6, P0.y * 1e6)
ax.set_xlabel(r"$x$ (µm)")
ax.set_ylabel(r"$y$ (µm)")
G

# Run Impact-T

Make a function to compare variations of the space charge calc. 

In [ ]:
def make_impact(
    initial_particles,
    space_charge_on=False,
    point_to_point=True,
):
    # Make Impact object
    I = Impact(IMPACT_IN)
    I.header["Nx"] = 16  # Larger numbers seem to give errors below.
    I.header["Ny"] = 16
    I.header["Nz"] = 64

    I.initial_particles = initial_particles
    if space_charge_on:
        I.total_charge = initial_particles["charge"]
    else:
        I.total_charge = 0

    # Patch in point_to_point_spacecharge

    if point_to_point:
        ELE = {
            "type": "point_to_point_spacecharge",
            "cutoff_radius": 2.8179e-15,  # classical electron radius
            "name": "point_to_point_calc",
        }
    else:
        ELE = I.lattice[0]
    I.input["lattice"] = [ELE, I.lattice[2], I.lattice[-2]]

    I.header["Dt"] = 1e-13
    I.stop = 0.15

    I.ele["APEX_GUN"]["rf_frequency"] = 1  # Like a DC gun
    I.ele["APEX_GUN"]["rf_field_scale"] = 0.4e6

    return I


I0 = make_impact(P0, space_charge_on=False)
I1 = make_impact(P0, space_charge_on=True)
I2 = make_impact(P0, space_charge_on=True, point_to_point=False)


I1.lattice

# Run

In [ ]:
%%time
I0.verbose = False
I0.run()

In [ ]:
%%time
# Warning: The # of electrons divided by the number of processors should be an integer.
I1.numprocs = 2
I1.verbose = False
I1.run()

In [ ]:
%%time
I2.numprocs = 2
I2.verbose = False
I2.run()

In [ ]:
I1.path

In [ ]:
!cat {I1.path}/ImpactT.in

In [ ]:
I1.plot(y2="mean_kinetic_energy")

# Compare space charge off, on

In [ ]:
Poff = I0.particles["final_particles"]
Pon = I1.particles["final_particles"]
PonIGF = I2.particles["final_particles"]

In [ ]:
def compare_stats(k2, k1="mean_z", scale1=1, scale2=1):
    fig, ax = plt.subplots()
    for I, label in (
        (I1, "point-to-point SC"),
        (I2, "IGF SC"),
        (I0, "SC off"),
    ):
        ax.plot(I.stat(k1) * scale1, I.stat(k2) * scale2, label=label)
    ax.set_xlabel(k1)
    ax.set_ylabel(k2)
    plt.legend()


compare_stats("sigma_x")

In [ ]:
I1.output["stats"].keys()

In [ ]:
compare_stats("sigma_gamma", scale2=mec2)
plt.ylim(0, 1)
plt.xlabel(r"$z$ (m)")
plt.ylabel(r"$\sigma_E$ (eV)")

In [ ]:
def compare_particles(k1, k2, scale1=1, scale2=1, units1="", units2=""):
    fig, ax = plt.subplots()
    for p, label in (
        (Pon, "point-to-point SC"),
        (PonIGF, "IGF SC"),
        (Poff, "SC off"),
    ):
        ax.scatter(p[k1] * scale1, p[k2] * scale2, marker=".", label=label)
    ax.set_xlabel(k1 + units1)
    ax.set_ylabel(k2 + units2)
    plt.legend()


compare_particles(
    "x",
    "y",
    scale1=1e6,
    scale2=1e6,
    units1=" (µm)",
    units2=" (µm)",
)

In [ ]:
compare_particles("z", "energy")

In [ ]:
Poff.plot("delta_z", "delta_energy", bins=100)

In [ ]:
Pon.plot("delta_z", "delta_energy", bins=100)